# Data Analysis Notebook

This notebook demonstrates how to analyze quadcopter simulation data and logs.

## Setup

First, let's import the necessary libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from pathlib import Path

from quadcopter.dynamics import QuadState
from quadcopter.simulation import simulate
from quadcopter.controllers import PIDController, PositionController
from quadcopter.logging import simulate_with_logging
from quadcopter.plotting import plot_trajectory, plot_frequency_analysis

print("Libraries imported successfully")

## Run Simulation with Logging

Let's run a simulation and generate some data to analyze:

In [ ]:
# Create a position controller
position_controller = PositionController(
    x_pid=PIDController(kp=2.0, ki=0.1, kd=0.5),
    y_pid=PIDController(kp=2.0, ki=0.1, kd=0.5),
    z_pid=PIDController(kp=4.0, ki=0.2, kd=1.0),
    target_pos=np.array([1.0, -1.0, 2.0])
)

# Set up initial state
initial_state = QuadState(
    pos=np.array([0.0, 0.0, 0.0]),
    vel=np.array([0.0, 0.0, 0.0]),
    quat=np.array([1.0, 0.0, 0.0, 0.0]),
    ang_vel=np.array([0.0, 0.0, 0.0])
)

# Run simulation with logging
log = simulate_with_logging(
    duration=10.0,
    dt=0.02,
    controller=position_controller,
    initial_state=initial_state,
    method="rk4"
)

print(f"Simulation completed with {len(log.times)} data points")

## Export Data

Let's export the data in different formats:

In [ ]:
# Create a directory for our data
data_dir = Path("analysis_data")
data_dir.mkdir(exist_ok=True)

# Export to CSV
csv_file = data_dir / "simulation_data.csv"
log.save_csv(csv_file)
print(f"Data exported to {csv_file}")

# Export to JSON
json_file = data_dir / "simulation_data.json"
log.save_json(json_file)
print(f"Data exported to {json_file}")

# Export to MATLAB (if scipy is available)
try:
    mat_file = data_dir / "simulation_data.mat"
    log.save_matlab(mat_file)
    print(f"Data exported to {mat_file}")
except:
    print("Skipping MATLAB export (scipy not available)")

## Load and Analyze Data

Let's load the CSV data and perform some analysis:

In [ ]:
# Load CSV data
df = pd.read_csv(csv_file)
print("Data loaded successfully")
print(f"Shape: {df.shape}")
print("\nFirst few rows:")
print(df.head())

print("\nColumn names:")
print(df.columns.tolist())

## Statistical Analysis

Let's perform some statistical analysis on the data:

In [ ]:
# Basic statistics
print("Basic Statistics:")
print(df.describe())

# Position statistics
print("\nPosition Statistics:")
pos_cols = ['pos_x', 'pos_y', 'pos_z']
print(df[pos_cols].describe())

# Velocity statistics
print("\nVelocity Statistics:")
vel_cols = ['vel_x', 'vel_y', 'vel_z']
print(df[vel_cols].describe())

# Control statistics
print("\nControl Statistics:")
control_cols = ['control_1', 'control_2', 'control_3', 'control_4']
print(df[control_cols].describe())

## Visualization

Let's create some visualizations of the data:

In [ ]:
# Extract data for plotting
t = df['time'].values
positions = df[['pos_x', 'pos_y', 'pos_z']].values
velocities = df[['vel_x', 'vel_y', 'vel_z']].values
controls = df[['control_1', 'control_2', 'control_3', 'control_4']].values

# Create states array for plotting functions
states = np.column_stack([
    positions,
    velocities,
    np.tile([1.0, 0.0, 0.0, 0.0], (len(t), 1)),  # Quaternion (identity for simplicity)
    np.zeros((len(t), 3))  # Angular velocity
])

# Plot trajectory
plot_trajectory(t, states, controls)

# Plot frequency analysis
plot_frequency_analysis(t, positions, ['X Position', 'Y Position', 'Z Position'])

## Error Analysis

Let's analyze the control errors:

In [ ]:
# Define target positions
target_pos = np.array([1.0, -1.0, 2.0])

# Calculate position errors
position_errors = positions - target_pos

# Plot position errors
plt.figure(figsize=(12, 8))
plt.subplot(3, 1, 1)
plt.plot(t, position_errors[:, 0])
plt.ylabel('X Error [m]')
plt.title('Position Errors')
plt.grid(True, alpha=0.3)

plt.subplot(3, 1, 2)
plt.plot(t, position_errors[:, 1])
plt.ylabel('Y Error [m]')
plt.grid(True, alpha=0.3)

plt.subplot(3, 1, 3)
plt.plot(t, position_errors[:, 2])
plt.xlabel('Time [s]')
plt.ylabel('Z Error [m]')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate error metrics
rmse = np.sqrt(np.mean(position_errors**2, axis=0))
mean_abs_error = np.mean(np.abs(position_errors), axis=0)
max_error = np.max(np.abs(position_errors), axis=0)

print("Error Metrics:")
print(f"RMSE - X: {rmse[0]:.4f} m, Y: {rmse[1]:.4f} m, Z: {rmse[2]:.4f} m")
print(f"Mean Absolute Error - X: {mean_abs_error[0]:.4f} m, Y: {mean_abs_error[1]:.4f} m, Z: {mean_abs_error[2]:.4f} m")
print(f"Max Error - X: {max_error[0]:.4f} m, Y: {max_error[1]:.4f} m, Z: {max_error[2]:.4f} m")

## Performance Analysis

Let's analyze the control performance:

In [ ]:
# Calculate control effort
control_effort = np.sum(controls**2, axis=1)

# Plot control effort
plt.figure(figsize=(10, 6))
plt.plot(t, control_effort)
plt.xlabel('Time [s]')
plt.ylabel('Control Effort')
plt.title('Control Effort Over Time')
plt.grid(True, alpha=0.3)
plt.show()

# Calculate total control effort
total_control_effort = np.sum(control_effort)
mean_control_effort = np.mean(control_effort)

print("Control Effort Metrics:")
print(f"Total Control Effort: {total_control_effort:.2f}")
print(f"Mean Control Effort: {mean_control_effort:.2f}")

# Analyze convergence
final_position = positions[-1]
position_error_norm = np.linalg.norm(final_position - target_pos)

print(f"\nConvergence Metrics:")
print(f"Final Position: [{final_position[0]:.3f}, {final_position[1]:.3f}, {final_position[2]:.3f}]")
print(f"Target Position: [{target_pos[0]:.3f}, {target_pos[1]:.3f}, {target_pos[2]:.3f}]")
print(f"Final Position Error: {position_error_norm:.4f} m")

## Summary

This notebook demonstrated:
1. How to run simulations with logging
2. How to export data in multiple formats
3. How to load and analyze simulation data
4. How to perform statistical analysis
5. How to visualize simulation results
6. How to analyze control errors and performance

This type of analysis is useful for:
- Tuning controller parameters
- Comparing different control strategies
- Generating reports for academic publications
- Validating simulation results